In [1]:
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
plt.style.use("ggplot")
# plot inline
%matplotlib inline

In [2]:
drug_class = pd.read_pickle('/home/hermuba/data/drug/anno_abx_class')

In [3]:
dummy_drug = pd.get_dummies(drug_class, columns = ['category'])

In [4]:
dummy_drug_index = dummy_drug.columns[2:]

In [5]:
amr_data = pd.read_pickle('/home/hermuba/data/annotated_RIS/anno_sps_df')
amr_data['Antibiotic'] = amr_data['Antibiotic'].str.lower()


# some numeric operation
import math
amr_data = amr_data.loc[amr_data['Measurement Value'] != 'nan']
amr_data = amr_data.loc[~amr_data['Measurement Value'].str.contains('/')]
amr_data['log_mic'] = pd.to_numeric(amr_data['Measurement Value']).apply(lambda x:np.log(x)/np.log(2))
amr_data = amr_data.loc[amr_data['log_mic'].apply(np.isfinite)]
amr = amr_data[['Genome ID', 'Antibiotic', 'Species', 'Resistant Phenotype', 'log_mic']]

cluster = pd.read_pickle('/home/hermuba/data/genePredicted/cdhit/ec0102_df')
card = pd.read_pickle('/home/hermuba/data/aro_pattern_df')

# look at the data:
X1 = norm_tmacc contains the chemical structure; pca_tmacc # index = Antibiotics

X2 = cluster (cdhit);card # index = Genome ID

Y = amr_data contains RIS data

# The Interaction
What we want to do is to know the relation between X1 and other Xs and how that affects the value Y

In [8]:
def merge_all(X1,X2,Y):
    df = Y.merge(X1, left_on = 'Genome ID', right_index = True)
    print(X2.columns)
    df = df.merge(X2, left_on = 'Antibiotic', right_on = 'drug')
    return(df)

# pearson correlation works really bad with sparse matrix
1. try mutual information instead.
2. fischer's exact test

In [29]:
import scipy.stats as stats
def contigency(df): # df contains three columns: col1 from gene; col2 from abx; col3 for resistant phenotype
    r = df.loc[df['Resistant Phenotype'] == 'Resistant'] # select those for resistant
    print(r.columns[:2])
    table = r.groupby(list(r.columns[0:2])).count()
    return(table)
def fisher(df, cset1, cset2):
    itxn_corr = []
    itxn_features = pd.DataFrame()
    for col1 in cset1:
        for col2 in cset2:
            t = contigency(df[[col1, col2, 'Resistant Phenotype']])
            

In [43]:
stats.fisher_exact(contigency(clu_drug[['Cluster 6', 'category_aminoglycosides', 'Resistant Phenotype']]))

Index(['Cluster 6', 'category_aminoglycosides'], dtype='object')


ValueError: The input `table` must be of shape (2, 2).

In [37]:
stats.fisher_exact([[65,7],[143,8]])

(0.51948051948051943, 0.25561135591937495)

In [25]:
def simple_correlation(df, column_sets):
    sim_corr = []
    for col in column_sets:
        corr = df['log_mic'].corr(df[col])
        sim_corr.append(corr)
    return(np.array(sim_corr))

In [26]:
def itxn_correlation(df, cset1, cset2):
    itxn_corr = []
    itxn_features = pd.DataFrame()
    for col1 in cset1:
        for col2 in cset2:
            itxn = df[col1].multiply(df[col2])
            
            corr = df['log_mic'].corr(itxn)
            itxn_corr.append(corr)
            if corr > 0.6:
                itxn_features[str(col1)+':'+str(col2)] = itxn
    return(np.array(itxn_corr), itxn_features)

In [10]:
clu_drug = merge_all(cluster,dummy_drug,amr)

Index(['drug', 'order', 'category_aminoglycosides',
       'category_b_lactam_inhibitor', 'category_carbapenems',
       'category_cephems', 'category_folate_path', 'category_fosfomycins',
       'category_licosamide', 'category_lipopeptides', 'category_macrolides',
       'category_monobactams', 'category_nitrofurans', 'category_penicillins',
       'category_phenicols', 'category_pleuromutilin', 'category_quinolones',
       'category_tetracyclines'],
      dtype='object')


In [29]:
itxn_correlation(clu_drug, dummy_drug_index, cluster.columns)

(array([ nan,  nan,  nan, ...,  nan,  nan,  nan]), Empty DataFrame
 Columns: []
 Index: [])

In [36]:
amr.columns


Index(['Genome ID', 'Antibiotic', 'Species', 'Resistant Phenotype', 'log_mic'], dtype='object')

In [64]:
dummy_drug

,drug,order,category_aminoglycosides,category_b_lactam_inhibitor,category_carbapenems,category_cephems,category_folate_path,category_fosfomycins,category_licosamide,category_lipopeptides,category_macrolides,category_monobactams,category_nitrofurans,category_penicillins,category_phenicols,category_pleuromutilin,category_quinolones,category_tetracyclines
0,nitrofurantoin,12.0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1,ciprofloxacin,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
2,levofloxacin,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,ofloxacin,10.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,trimethoprim/sulfamethoxazole,11.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
5,sulfonamides,11.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
6,trimethoprim,11.0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
7,chloramphenicol,6.0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
8,fosfomycin,13.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
9,netilmicin,8.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
